# Criando um csv com os arquivo raw
Para reduzir a complexidade das consultas e retirar informações sensíveis.

In [ ]:
import pandas as pd 


In [3]:
df = pd.read_csv('data/INFLUD25-07-07-2025.csv',sep = ';')

C:\Users\rafik\AppData\Local\Temp\ipykernel_26672\2456070258.py:1: DtypeWarning: Columns (14,92,94,185,186,188,189) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/INFLUD25-07-07-2025.csv',sep = ';')


In [4]:
df.head()

,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
0,31735451969802,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,31735516593521,2024-12-29,1,2024-12-29,1,SP,GVE VIII MOGI DAS CRUZES,1333.0,GUARULHOS,351880,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
2,31735544021940,2024-12-29,1,2024-12-29,1,SP,GVE XXIX SAO JOSE DO RIO PRETO,1354.0,SAO JOSE DO RIO PRETO,354980,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
3,31735559043222,2024-12-30,1,2024-12-29,1,AC,REGIONAL DO BAIXO ACRE,1938.0,RIO BRANCO,120040,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
4,31735560267779,2024-12-30,1,2024-12-29,1,PB,I NRS JOAO PESSOA,1377.0,JOAO PESSOA,250750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [19]:
df_parcial = df[['DT_SIN_PRI','CLASSI_FIN', 'EVOLUCAO', 'HOSPITAL', 'UTI','VACINA_COV', 'DT_NOTIFIC','SG_UF', 'DT_EVOLUCA' ]].copy()

In [20]:
df_parcial = df_parcial.dropna()

In [27]:
df_parcial['DT_SIN_PRI'] = pd.to_datetime(df_parcial['DT_SIN_PRI'], format='%d/%m/%Y', errors='coerce')
df_parcial['DT_NOTIFIC'] = pd.to_datetime(df_parcial['DT_NOTIFIC'], format='%d/%m/%Y', errors='coerce')
df_parcial['DT_EVOLUCA'] = pd.to_datetime(df_parcial['DT_EVOLUCA'], format='%d/%m/%Y', errors='coerce')

In [ ]:
df_parcial["data"] = df_parcial["DT_SIN_PRI"].fillna(df_parcial["DT_NOTIFIC"])

df_parcial["is_obito"] = df_parcial["EVOLUCAO"] == 2
df_parcial["is_uti"] = df_parcial["UTI"] == 1
df_parcial["is_hospitalizado"] = df_parcial["HOSPITAL"] == 1
df_parcial["is_vacinado"] = df_parcial["VACINA_COV"] == 1

df_parcial = df_parcial[df_parcial["data"] >= pd.Timestamp.today() - pd.Timedelta(days=30)]

df_agg = df_parcial.groupby("data").agg(
    total_casos=("data", "count"),
    obitos=("is_obito", "sum"),
    hospitalizados=("is_hospitalizado", "sum"),
    uti=("is_uti", "sum"),
    vacinados=("is_vacinado", "sum")
).reset_index()

df_agg["media_movel_7d"] = df_agg["total_casos"].rolling(7).mean()


In [21]:
df_parcial.to_csv(('data/srag_parcial.csv'), index=False, sep=';')